In [2]:
from tqdm import tqdm
import numpy as np
import pickle
import openai
import sys
sys.path.insert(1, '../../model/')
import eval
openai.api_key = "<OPENAI-KEY>"

In [ ]:
DATASET = "concept_properties" # "feature_norms"

In [3]:
def get_prompts(prompt_type):
	noun2sent = {}
	with open(f"../data/datasets/{DATASET}/queries" + prompt_type + ".prop", "r") as f:
		for raw_data in f.readlines():
			noun = raw_data.split(" :: ")[0]
			sent = raw_data.split(" :: ")[1][:-1]
			noun2sent[noun] = sent
	return noun2sent

In [4]:
noun2prop = pickle.load(open(f"../data/datasets/{DATASET}/noun2property/noun2prop.p", "rb"))
candidate_adjs = []
for noun, props in noun2prop.items():
    candidate_adjs += props
candidate_adjs = list(set(candidate_adjs))

In [44]:
def get_gpt3_scores(sentence):
    response = openai.Completion.create(
                        engine="text-ada-001",
                        prompt=sentence,
                        temperature=1.0,
                        max_tokens=0,
                        top_p=1,
                        echo=True,
                        logprobs=1,
                        frequency_penalty=0,
                        presence_penalty=0
                    )
    tokens = dict(dict(dict(dict(response)['choices'][0]))['logprobs'])['tokens']
    logprobs = dict(dict(dict(dict(response)['choices'][0]))['logprobs'])['token_logprobs']
    list_of_tokens = []
    current_list = []
    for i, token in enumerate(tokens):
        if token != "\n":
            current_list.append((token, logprobs[i]))
        else:
            list_of_tokens.append(current_list)
            current_list = []
    list_of_tokens.append(current_list)
    scores = []
    for token_list in list_of_tokens:
        # for i, (token, prob) in enumerate(token_list):
        #     if token == " is" or token == " are" or token == " be":
        #         target_index = i + 1
        #         break
        current_scores = [s for t, s in token_list[target_index:]]
        current_scores = [s for _, s in token_list if s != None]
        scores.append(2 ** (-np.mean(current_scores)))
    return scores

In [ ]:
prompt_types = ["plural_most", "singular_can_be", "singular_usually", "singular_generally", "singular", "plural_all", "plural_can_be", "plural_generally", "plural_some", "plural_usually", "plural"]
for pt in prompt_types:
    noun2sent = get_prompts(prompt_type = pt)
    noun2predicts = {}
    for noun, sent in tqdm(noun2sent.items()):
        noun2predicts[noun] = []
        sentences = [sent.replace('[MASK]', adj) for adj in candidate_adjs]
        input2gpt3 = "\n".join(sentences)
        scores = get_gpt3_scores(input2gpt3)
        predicts = [(candidate_adjs[ind], float(scores[ind])) for ind in np.argsort(scores)]
        predicts.sort(key=lambda x: x[0])
        predicts.sort(key=lambda x: x[1])
        noun2predicts[noun] = [pred[0] for pred in predicts]
    eval.evaluate(noun2predicts, noun2prop)
    eval.evaluate_acc(noun2predicts, noun2prop)
    eval.evaluate_concretness(noun2predicts)